In [1]:
%load_ext autoreload
%autoreload 2

import os
import sys
sys.path.insert(1, '/home/cem/Documents/imps/src')

import numpy as np
import open3d as o3d
import torch
from torch.nn import functional as F
from matplotlib import cm
import matplotlib.pyplot as plt

from tqdm import tqdm

from imps.data import ScanNetScene, CLASS_NAMES
from imps.ndf.local_model import MiniNDF, NDF

SCENE_DIR = '/home/cem/Documents/datasets/ScanNet/scans/scene0000_00'

N_POINTS = int(2e4)
RESOLUTION = 125
SIGMAS = np.array([0.75, 0.45, 0.05])
DEVICE = 'cuda'

In [2]:
scene = ScanNetScene(SCENE_DIR)

voxel_grid, surface_points, vicinity_points, point_labels = scene.create_if_data(
    RESOLUTION, N_POINTS, SIGMAS, vicinity_ratio=1, o3d_format=True
)

In [3]:
o3d.visualization.draw_geometries([voxel_grid, vicinity_points])

In [9]:
# Now generate numeric data instead of visual
scene = ScanNetScene(SCENE_DIR)

voxel_grid, _, _, vicinity_points, vicinity_distances, _ = scene.create_if_data(
    RESOLUTION, N_POINTS, SIGMAS, vicinity_ratio=1
)

voxel_grid = torch.FloatTensor(voxel_grid).unsqueeze(0).to(DEVICE)

model = NDF()
model = model.cuda()
model.load_state_dict(torch.load('../../data/MiniNDF-11-scenes')['model_state_dict'])
model.eval();

In [10]:
def get_cube(cube_center, cube_size, n=10):
    x_min, x_max = cube_center[0]-cube_size/2, cube_center[0]+cube_size/2
    y_min, y_max = cube_center[1]-cube_size/2, cube_center[1]+cube_size/2
    z_min, z_max = cube_center[2]-cube_size/2, cube_center[2]+cube_size/2

    x_ = np.linspace(x_min, x_max, n)
    y_ = np.linspace(y_min, y_max, n)
    z_ = np.linspace(z_min, z_max, n)
    
    X, Y, Z = np.meshgrid(x_, y_, z_, indexing='ij')
    return np.vstack([X.ravel(), Y.ravel(), Z.ravel()]).T

def surface_grad(model, input, initial_points, num_steps, step_size):
    for param in model.parameters():
        param.requires_grad = False
            
    encoding = model.encoder(input)

    samples = torch.Tensor(initial_points.reshape(1, len(initial_points), 3)).float().to('cuda')
    samples.requires_grad = True

    sample_steps = np.zeros((num_steps, len(initial_points), 3))

    i = 0

    for j in range(num_steps):
        # Update this in NDF mocel
        df_pred = model.decoder(samples, *encoding)
        df_pred.sum().backward()
        
        gradient = samples.grad.detach()
        samples = samples.detach()
        df_pred = df_pred.detach()
        input = input.detach()
        
        samples = samples - step_size*(F.normalize(gradient, dim=2)*df_pred.reshape(-1, 1))

        samples = samples.detach()
        samples_cpu = samples.detach().cpu().numpy()[0]
        sample_steps[j] = samples_cpu
        
        samples.requires_grad = True
        
    return sample_steps

def generate_point_cloud(model, inputs, num_steps=10, num_points=900000, filter_val=0.009, threshold=0.025):
    for param in model.parameters():
        param.requires_grad = False

    sample_num = 20000
    samples_cpu = np.zeros((0, 3))
    samples = torch.rand(1, sample_num, 3).float().to(DEVICE) * 3 - 1.5
    samples.requires_grad = True

    encoding = model.encoder(inputs)

    i = 0
    while len(samples_cpu) < num_points:
        print('iteration', i)

        for j in range(num_steps):
            df_pred = torch.clamp(model.decoder(samples, *encoding), max=threshold)

            df_pred.sum().backward()

            gradient = samples.grad.detach()
            samples = samples.detach()
            df_pred = df_pred.detach()
            inputs = inputs.detach()
            samples = samples - F.normalize(gradient, dim=2) * df_pred.reshape(-1, 1)  # better use Tensor.copy method?
            samples = samples.detach()
            samples.requires_grad = True

        if not i == 0:
            samples_cpu = np.vstack((samples_cpu, samples[df_pred < filter_val].detach().cpu().numpy()))

        samples = samples[df_pred < 0.03].unsqueeze(0)
        indices = torch.randint(samples.shape[1], (1, sample_num))
        samples = samples[[[0, ] * sample_num], indices]
        samples += (threshold / 3) * torch.randn(samples.shape).to(DEVICE)  # 3 sigma rule
        samples = samples.detach()
        samples.requires_grad = True

        i += 1
        print(samples_cpu.shape)

    return samples_cpu

In [25]:
r1, r2 = scene.bounds
cube_size = 0.1
cube_x_ = np.arange(r1[0]+cube_size/2, r2[0]-cube_size/2, cube_size)
cube_y_ = np.arange(r1[1]+cube_size/2, r2[1]-cube_size/2, cube_size)
cube_z_ = np.arange(r1[2]+cube_size/2, r2[2]-cube_size/2, cube_size)
cube_X, cube_Y, cube_Z = np.meshgrid(cube_x_, cube_y_, cube_z_, indexing='ij')

cube_centers = np.vstack([cube_X.ravel(), cube_Y.ravel(), cube_Z.ravel()]).T 
all_steps = []

for cc in tqdm(cube_centers):
    init_points = get_cube(cc, cube_size)
    init_points = init_points[init_points[:, -1] < 0.05]
    surface_steps = surface_grad(model, voxel_grid, init_points, 5, 0.5)
    all_steps.append(surface_steps)
    
all_steps = np.concatenate(all_steps, axis=1)

100%|██████████████████████████████████████████████████████████████████| 243/243 [00:16<00:00, 14.50it/s]


In [26]:
import pickle
pickle.dump(all_steps, open('../../data/ndf_steps.pkl', 'wb'))

In [27]:
query_points = torch.FloatTensor(vicinity_points).unsqueeze(0).cuda()

with torch.no_grad():
    encoding = model.encoder(voxel_grid)
    pred_dist = model.decoder(query_points, *encoding)
    
pts = query_points.squeeze().detach().cpu().numpy()
dst = pred_dist.squeeze().detach().cpu().numpy()
dst_norm = (dst - dst.min()) / (dst.max() - dst.min())

viridis = cm.get_cmap('Reds')
vic_pcd = o3d.geometry.PointCloud()
vic_pcd.points = o3d.utility.Vector3dVector(pts)
vic_pcd.colors = o3d.utility.Vector3dVector(viridis(dst)[:, :-1])
    
o3d.visualization.draw_geometries([vic_pcd])

In [22]:
np.abs(dst - vicinity_distances).mean()*scene.scale

0.14207502427378035